<H1 style="color: black; text-aling: center; font-size: 25px; ">Functions for Tides, Shape and Rotation by Sofía Arboleda-Bolívar </H1>

<style>
        p {
            text-align: justify; line-height: 1.5;
        }
</style>

<p>In this notebook you'll find the functions used for the main code of the section Tides, Shape and Rotation. </p>

👉 😭 For details on the theory please refer directly to the book:
> Murray, C. D., & Dermott, S. F. (2000). Solar System Dynamics. Cambridge University Press.

<H1 style="color: black; text-aling: center; font-size: 15px; ">Constants </H1>

In [3]:
from astropy import constants as const

R_earth = float(const.R_earth.value) #Earth's radius
G = float(const.G.value)
g = float(const.g0.value)
au = float(const.au.value)
rho_earth = 5.51e3 #Earth density

m_earth = float(const.M_earth.value)
m_moon = 7.342e22 # Moon mass
a_em = 384400e3 # Mean Earth-Moon distance
m_sun = float(const.M_sun.value)

<H1 style="color: black; text-aling: center; font-size: 15px; ">Functions</H1>

In [4]:
#Legendre polynomial of degree 2 in cos(Psi)
def P_2(psi):
    r"""Returns the Legendre polynomial of 
        degree 2: P(cos(psi))
    
    Parameter
    ---------
    Psi: angle ψ is measured from the line joining the centres of the two bodies. 
    
    Returns
    -------
    P(cos(Psi)) : Legendre polynomial. 
    
    Equation
    -------
    :math:`\P_{2}(cos(\Psi))= \frac{1}{2}(3) `
    """
    f = (1/2)*(3*np.cos(psi)**2 - 1)
    return f

#Second component of the potential 
def V2(psi, V1, Rp, a):
    r"""Returns the second component of the potential defined in eq. 4.7
    
    Parameter
    ---------
    Psi: angle ψ is measured from the line joining the centres of the two bodies. 
    V1: First component of the potential (array with constant value)
    Rp: Radius of the planet
    a: Distance from center of one body to the other
    
    Returns
    -------
    V2 : Second component of the total potential. 
    
    Equation
    -------
    :math:`\V_{2}= V_1 \left(\frac{R_p}{a} cos(\Psi)\rigth) `
    """
    f = V1*(Rp/a)*np.cos(psi)
    return f

# Third component of the potential
def V3(psi, V1, Rp, a):
    r"""Returns the third component of the potential as defined in eq. 4.7
    
    Parameter
    ---------
    Psi: angle ψ is measured from the line joining the centres of the two bodies. 
    V1: First component of the potential (array with constant value)
    Rp: Radius of the planet
    a: Distance from center of one body to the other
    
    Returns
    -------
    V3 : Third component of the total potential. 
    
    Equation
    -------
    :math:`\V_{3}= V_1 \left(\frac{R_p}{a}\rigth)^2 \frac{1}{2} (3 cos^2(\Psi)-1) `
    """
    f = V1*(Rp/a)**2*P_2(psi)
    return f

# Total potential
def V_tot(V1, V2, V3):
    r"""Returns the total potential as defined in eq. 4.7
    
    Parameter
    ---------
    V1: First component of the potential (array with constant value)
    V2: Second component of the potential 
    V3: Third component of the potential 
    
    Returns
    -------
    V_total : Total potential. 
    
    Equation
    -------
    :math:`\V_{Tot}= V_1 + V_2 + V_3 `
    """
    f = V1 + V2 + V3
    return f


In [5]:
#Importing naif kernels needed
def naif_kernels(t):
    r"""This function is the download of the two naif kernels needed to 
        implement the Coordinate_transf function.
    """
    
    !wget -O earth_200101_990628_predict.bpc https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_200101_990628_predict.bpc
    !wget -O latest_leapseconds.tls https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/latest_leapseconds.tls
    
    return 

In [6]:
# Coordinate transformation 
def Coordinate_transf(P, S, t):
    
    r"""This routine is a facility to transform Earth coordinates
    of a given point at a given time to J2000 coordinates.
    
    It receives the coordinates of a point P, the indicator of the satellite
    (or celestial body to be used) as a list with a string and a Horizons 
    number indicator and the given date (in date time format) in which the 
    data is going to be taken.
    It returns both coordinates in ecuatorial coordinates in the J2000 system.
    
    *Documentation needed (kernels): earth_200101_990628_predict.bpc & latest_leapseconds.tls
    
    Parameter
    ---------
    P = array[lon[º], lat[º], alt[km]] 
    S = array['satellite_name',
         Horizons_id] 
    t = string("yyyy-mm-dd hh:mm:ss")
    
    Returns
    -------
    Coordinates of point p and point s: np.array([phiP, thetaP, phiS, thetaS]) 
    """
    lon, lat, alt = P[0], P[1], P[2]  
    
    ind = S[1]
    
    t = str(t)

    #Earth's oblateness
    f = 1/298.257223563
    
    #Degrees to radians
    lon = np.radians(lon)
    lat = np.radians(lat)

    rP_earth = spy.georec(lon,lat,alt,6371,f)

    
    #UTC to Ephemeris Time
    et = spy.utc2et(t)
    
    #Conversion matrix from International Terrestrial Reference Frame to J200
    R_earth2J2000 = spy.pxform("ITRF93","J2000",et)
    
    #Transformation of coordinates with the matrix 
    rP_J2000 = spy.mxv(R_earth2J2000,rP_earth)

    #From rectangular coordinates to latitudinal coordinates
    d, lon, lat = spy.reclat(rP_J2000) 
    phiP = lon
    thetaP = np.pi/2 - lat #Colatitude

    jd = spy.unitim(et,"ET","JDTDT")

    satellite_data = Horizons(id=ind, location="@3", epochs=jd).ephemerides().to_pandas()
    phiS = np.radians(float(satellite_data.RA))
    thetaS = np.radians(float(90 - satellite_data.DEC))
    
    return np.array([phiP, thetaP, phiS, thetaS])

In [2]:
# Cosine of the angle ψ between the points of coordinates from last function
def cosPsi(coord):
    r"""Receives coordinates as given from Coordinate_transf 
         and returns cosine of ψ as defined in eq. 4.15
         
    Parameter
    ---------
    Coord: np.array([phiP, thetaP, phiS, thetaS])
    
    Returns
    -------
    cos(Psi) 
    
    Equation
    -------
    :math:`cos(\Psi) = cos(\theta_P) cos(\theta_M) + 
        sin(\theta_P)sin(\theta_M)cos(\Phi_P - \Phi_M)`
    """
    phiP = coord[0]
    thetaP = coord[1]
    phiS = coord[2]
    thetaS = coord[3]
    cosine = np.cos(thetaP)*np.cos(thetaS) + np.sin(thetaP)*np.sin(thetaS)*np.cos(phiP-phiS)
    
    return cosine 

# Amplitude of the equilibrium tide
def dseta(Rp, a, ms, mp):
    r"""This function calculates the amplitude of the equilibrium tide
        as defined in eq. 4.13.
        
    Parameter
    ---------
    Rp: Radius of the planet
    a: Semi-major axis of the satellite's orbit
    ms: Satellite's mass
    mp: Planet's mass
    
    Returns
    -------
    dseta: Amplitude of the equilibrium tide
    
    Equation
    -------
    :math:`\dseta= \frac{m_s}{m_p} \left(\frac{R_p}{a}\rigth)^3 R_P `
    """
    f = Rp*(ms/mp)*(Rp/a)**3
    return f

# Surface gravity of the main body 
def g_f(Rp, mp):
    r"""This function calculates the surface gravity of the planet
        as defined in eq. 4.14.
        
    Parameter
    ---------
    Rp: Radius of the planet
    mp: Planet's mass
    
    Returns
    -------
    g: Gravitational acceleration
    
    Equation
    -------
    :math:`g = \frac{m_p}{R_P}^2 G `
    """
    f = (G*mp/Rp**2)
    return f

def Tide(cos_Psi, ms, mp, Rp, a):
    r"""This function gives the tide raising part of the
        potential as defined in eq. 4.12.
        
    Parameter
    ---------
    cos_Psi: Result from the implementation of the function 
        cosPsi of the given coordinates 
    Rp: Radius of the planet
    mp: Planet's mass
    ms: Satellite's mass
    a: Semi-major axis of the satellite's orbit
    
    Returns
    -------
    V3: tide raising part of the potential
    
    Equation
    -------
    :math:`V_3 = \dseta g P_2(cos(\Psi)) `
    """
    f = -dseta(Rp, a, ms, mp)*g_f(Rp, mp)*P_2(cos_Psi)
    return f

In [6]:
def v_int(r, theta, C, gamma, epsilon_2):
    r"""
        
    Parameter
    ---------
    
    Returns
    -------
    
    
    Equation
    -------
    :math:`V_3 = \dseta g P_2(cos(\Psi)) `
    """
    f = -4/3*np.pi*C**3*gamma*G*(((3*C**2-r**2)/(2*C**3))+(3*r**2*epsilon_2*P_2(theta))/(5*C**3))
    return f

def v_ext(r, theta, C, gamma, epsilon_2):
    r"""
        
    Parameter
    ---------
    
    Returns
    -------
    
    
    Equation
    -------
    :math:`V_3 = \dseta g P_2(cos(\Psi)) `
    """
    f = -4/3*np.pi*C**3*gamma*G*((1/r)+ (3*C**2*epsilon_2*P_2(theta))/(5*r**3))
    return f


def v_int_homo(r, C, gamma):
    r"""
        
    Parameter
    ---------
    
    Returns
    -------
    
    
    Equation
    -------
    :math:`V_3 = \dseta g P_2(cos(\Psi)) `
    """
    f = -2/3*np.pi*gamma*G*(3*C**2-r**2)
    return f
    
def v_ext_homo(r, C, gamma):
    r"""
        
    Parameter
    ---------
    
    Returns
    -------
    
    
    Equation
    -------
    :math:`V_3 = \dseta g P_2(cos(\Psi)) `
    """
    f = -4/3*np.pi*gamma*G*C**3/r
    return f

In [7]:
def Rcb(S2, Psi, A):
    return A*(1+S2*P_2(np.cos(Psi)))

def Ros(T2, Psi, B):
    return B*(1+T2*P_2(np.cos(Psi)))

def S2(A, F, zeta_c, mu_):
    return (F/A)*(5/2)*zeta_c/(1+mu_)

def T2(B, H, zeta):
    return (H/B)*(5/2)*zeta

def F(mu_, sigma, rho, A, B):
    alpha = 1 + (5/2)*(rho/sigma)*(A/B)**3*(1-sigma/rho)
    return ((1 + mu_)*(1 - sigma/rho)*(1 + 3/(2*alpha)))/(1+mu_-(sigma/rho)+(3*alpha/(2*rho))*(1-(sigma/rho))-(9/(4*alpha))*(A/B)**5*(1-(sigma/rho))**2)

def H(mu_, sigma, rho, A, B, F):
    delta = (A/B)**3*(1 - sigma/rho)
    return 2*rho/(5*rho)*((1 + mu_ + (3/2)*(A/B)**2*F*delta)/((1 + mu_)*(delta + 2*sigma/(5*rho))))